In [1]:
import os
import numpy as np
import sagemaker
from sagemaker.session import s3_input

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
hyperparams={'epochs': 20,
                 'learning-rate': 0.01,
                 'batch-size': 256,
                 'weight-decay': 2e-4,
                 'momentum': 0.9,
                 'optimizer': 'sgd'}

In [3]:
from sagemaker.tensorflow import TensorFlow

output_path = 's3://sagemaker-container-webinar/'
tf_estimator = TensorFlow(entry_point='cifar10-training-script-sagemaker.py', 
                          source_dir='code',
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.13', 
                          py_version='py3',
                          script_mode=True,
                          output_path=output_path,
                          hyperparameters=hyperparams)

#train_instance_type = 'ml.p3.2xlarge'

In [4]:
train_path = 's3://sagemaker-container-webinar/cifar10-dataset/train'
val_path = 's3://sagemaker-container-webinar/cifar10-dataset/validation'
eval_path = 's3://sagemaker-container-webinar/cifar10-dataset/eval'

tf_estimator.fit({'training': train_path,'validation': val_path,'eval': eval_path})

2019-08-07 06:54:30 Starting - Starting the training job...
2019-08-07 06:54:33 Starting - Launching requested ML instances......
2019-08-07 06:55:39 Starting - Preparing the instances for training...
2019-08-07 06:56:27 Downloading - Downloading input data...
2019-08-07 06:56:40 Training - Downloading the training image...
2019-08-07 06:57:31 Training - Training image download completed. Training in progress..
2019-08-07 06:57:34,169 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-08-07 06:57:34,577 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "training": "/opt/ml/input/data/training",
        "validation": "/opt/ml/input/data/validation"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperpa

In [5]:
predictor = tf_estimator.deploy(initial_instance_count=1, instance_type='ml.c5.large')
#ml.c5.large

---------------------------------------------------------------------------!

In [6]:
print(predictor.endpoint)
print(predictor.content_type)
print(predictor.serializer)

tensorflow-training-2019-08-07-06-54-30-212
application/json


In [7]:
labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
images = []
for entry in os.scandir('sample-img'):
    if entry.is_file() and entry.name.endswith("png"):
        images.append('sample-img/' + entry.name)

In [8]:
import cv2
def image_file_to_tensor(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.asarray(image)
    image = np.expand_dims(image, axis=0)
    return image

def get_prediction(file_path):
    probs = predictor.predict(image_file_to_tensor(file_path))['predictions']
    return labels[np.argmax(probs, axis=1)[0]]

predictions = [get_prediction(image) for image in images]
print(' ')
for x,y in zip(images, predictions):
    print(x,y)

 
sample-img/1009_frog.png frog
sample-img/1008_truck.png cat
sample-img/1014_cat.png cat
sample-img/1003_deer.png deer
sample-img/1004_ship.png ship
sample-img/1000_dog.png dog
sample-img/1005_automobile.png automobile
sample-img/1038_bird.png bird
sample-img/1001_airplane.png airplane
sample-img/1037_horse.png bird


In [9]:
#sample-img/1001_airplane.png
#sample-img/1009_frog.png
import numpy as np
from PIL import Image

def get_prediction(file_path):
    img = Image.open(file_path) 
    image_content = np.reshape(np.array(img),(1,32,32,3))
    return labels[np.argmax(predictor.predict(image_content)['predictions'], axis=1)[0]]

predictions = [get_prediction(image) for image in images]

print(' ')
for x,y in zip(images, predictions):
    print(x,y)

 
sample-img/1009_frog.png frog
sample-img/1008_truck.png cat
sample-img/1014_cat.png cat
sample-img/1003_deer.png deer
sample-img/1004_ship.png ship
sample-img/1000_dog.png dog
sample-img/1005_automobile.png automobile
sample-img/1038_bird.png bird
sample-img/1001_airplane.png airplane
sample-img/1037_horse.png bird


In [ ]:
tf_estimator.delete_endpoint()

In [28]:
# import boto3
# import json
# import base64
# import requests

# runtime = boto3.Session().client(service_name='runtime.sagemaker')

# file_name = 'sample-img/1000_dog.png'

# with open(file_name, 'rb') as f:
#     payload = bytearray(f.read())
    
# encoded_image = base64.b64encode(payload).decode('utf-8')
# instance = [{"b64": encoded_image}]

# response = runtime.invoke_endpoint(EndpointName=predictor.endpoint, 
#                                    ContentType='application/json', 
#                                    Body=json.dumps({"instances": instance}))
# result = response['Body'].read()
# # result will be in json format and convert it to ndarray
# result = json.loads(result)
# # the result will output the probabilities for all classes
# # find the class with maximum probability and print the class index
# index = np.argmax(result)


In [ ]:
# from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# hyperparameter_ranges = {
#     'epochs':        IntegerParameter(20, 100),
#     'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
#     'batch-size':    IntegerParameter(32, 1024),
#     'dense-layer':   IntegerParameter(128, 1024),
#     'dropout':       ContinuousParameter(0.2, 0.6)
# }

# objective_metric_name = 'val_acc'
# objective_type = 'Maximize'
# metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

# tuner = HyperparameterTuner(tf_estimator,
#                             objective_metric_name,
#                             hyperparameter_ranges,
#                             metric_definitions,
#                             max_jobs=10,
#                             max_parallel_jobs=2,
#                             objective_type=objective_type)